In [5]:
import os
import numpy as np
import trimesh
from scipy.ndimage import zoom
from tqdm import tqdm
import kagglehub

In [7]:
# Download latest version
path = kagglehub.dataset_download("balraj98/modelnet10-princeton-3d-object-dataset")

print("Path to dataset files:", path)

Resuming download from 7340032 bytes (469207757 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/balraj98/modelnet10-princeton-3d-object-dataset?dataset_version_number=1 (7340032/476547789) bytes left.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 454M/454M [14:46<00:00, 529kB/s]

Extracting files...


Path to dataset files: /Users/rezasarkhosh/.cache/kagglehub/datasets/balraj98/modelnet10-princeton-3d-object-dataset/versions/1


In [11]:
dataset_root = os.path.join(path, "ModelNet10") 
selected_classes = ['chair', 'table', 'sofa', 'monitor', 'desk', 'bed', 'bathtub', 'night_stand', 'toilet', 'dresser']
voxel_dim = 32

In [12]:
def mesh_to_voxel(path_to_off_file, voxel_dim=32):
    mesh = trimesh.load(path_to_off_file)
    voxelized = mesh.voxelized(pitch=mesh.scale / voxel_dim)
    voxel_matrix = voxelized.matrix.astype(np.float32)

    
    voxel_matrix = zoom(voxel_matrix, (
        voxel_dim / voxel_matrix.shape[0],
        voxel_dim / voxel_matrix.shape[1],
        voxel_dim / voxel_matrix.shape[2]
    ))

    return voxel_matrix

In [13]:
def load_voxel_dataset(dataset_path, voxel_dim=32, selected_classes=None):
    X, y = [], []
    if selected_classes is None:
        selected_classes = sorted(os.listdir(dataset_path))
    
    label_map = {cls_name: idx for idx, cls_name in enumerate(selected_classes)}

    for cls_name in tqdm(selected_classes, desc="Classes"):
        class_path = os.path.join(dataset_path, cls_name, "train")
        if not os.path.exists(class_path):
            print(f"❌ : {class_path}")
            continue
        
        file_list = [f for f in os.listdir(class_path) if f.endswith('.off')]
        for file_name in tqdm(file_list, desc=f"{cls_name}", leave=False):
            file_path = os.path.join(class_path, file_name)
            try:
                voxel = mesh_to_voxel(file_path, voxel_dim)
                X.append(voxel)
                y.append(label_map[cls_name])
            except Exception as e:
                print(f"⚠️ {file_path}: {e}")
    
    X = np.array(X)[..., np.newaxis]
    y = np.array(y)
    return X, y, label_map

In [14]:
X, y, label_map = load_voxel_dataset(dataset_root, voxel_dim, selected_classes)

np.save("X_voxel_full.npy", X)
np.save("y_voxel_full.npy", y)

print("✅ ")
print("🔢 ", X.shape, y.shape)
print("🏷️ ", label_map)

chair:  12%|████████████████                                                                                                                 | 111/889 [00:22<02:56,  4.42it/s]


chair:  25%|████████████████████████████████▊                                                                                                | 226/889 [00:48<02:43,  4.07it/s]


chair:  38%|████████████████████████████████████████████████▊                                                                                | 336/889 [01:13<01:28,  6.24it/s]


chair:  51%|█████████████████████████████████████████████████████████████████▉                                                               | 454/889 [01:41<01:20,  5.40it/s]


chair:  64%|█████████████████████████████████████████████████████████████████████████████████▉                                               | 565/889 [02:08<02:17,  2.36it/s]


...

toilet:  79%|█████████████████████████████████████████████████████████████████████████████████████████████████████▌                          | 273/344 [00:47<00:14,  4.76it/s]


dresser:  20%|██████████████████████████▏                                                                                                     | 41/200 [00:14<01:57,  1.36it/s]


dresser:  74%|█████████████████████████████████████████████████████████████████████████████████████████████▎                                 | 147/200 [00:57<00:29,  1.79it/s]


Classes: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [19:50<00:00, 119.06s/it]


✅ آماده‌سازی دیتا تمام شد.
🔢 شکل داده‌ها: (3991, 32, 32, 32, 1) (3991,)
🏷️ لیبل‌ها: {'chair': 0, 'table': 1, 'sofa': 2, 'monitor': 3, 'desk': 4, 'bed': 5, 'bathtub': 6, 'night_stand': 7, 'toilet': 8, 'dresser': 9}
